1. First create <b>`Dockerfile`</b> file

In [ ]:
                                                                                         # Dockerfile
# Bring Base Image From The Docker Hub
FROM python:3.7-alpine

# Name Of The Code Maintainer & Email 
MAINTAINER Saifullah al Mujahid <mujahid7292@gmail.com>

# Let set python unbuffered environment variable.
ENV PYTHONUNBUFFERED 1
# What this environment variable does, is that it tell python
# to run in unbuffered mode. This is recommended when running 
# python inside docker container. The reason for this, is that 
# it does not allow python to buffer the output. So it reduce
# the error in running python inside docker image.

# We are going to store our projects dependencies in the
# ./requirements.txt file.
COPY ./requirements.txt  /requirements.txt
# This command will copy this file to inside the docker image
# in the following '/' directory.

# Now we will install all of the dependencies of our project.
RUN pip install -r /requirements.txt

# We will create a directory(/app) within our docker image, that we can use
# to store our application source code.
RUN mkdir /app
# Create a directory/folder named 'app' inside our docker image
WORKDIR /app
# Switched to this 'app' directory as a default directory.
# Any application that we run inside our docker container, will start
# running from this location.
COPY ./app /app
# Copy all the source code from our local machine / 'app' folder to the directory
# inside docker container which is also 'app' folder.

# We will create an user who will run our application using docker.
RUN adduser -D mujahid7292
# -D = Create an user without admin privilage. That means this user will only be able 
# to run this docker image & will not get any root access.
# mujahid7292 = Name of the user.
USER mujahid7292
# Now docker will switched to this 'mujahid7292' user.


2. Create <b>`requirements.txt`</b> file

In [ ]:
                                                                                     #requirements.txt
Django>=2.1.4,<2.2.0
djangorestframework>=3.9.0,<3.10.0

3. Create <b>`docker-compose.yml`</b> file

In [ ]:
                                                                                # docker-compose.yml
version: "3"
# docker-compose version number. 3 is the lattest version

# Now we will define services that make up our application
services:
  app: # This is the name of our services
    build:
      context: . # This will set the build to our current directory
    ports:
      - "8000:8000"
      # First 8000, means we are exposing port 8000 in our machine.
      # Second 8000, means we are exposing port 8000 in our docker image.
    volumes:
    # Volumes allows us to get the update (that we make to our project) into our docker
    # image in real time. So this section will maps a volume from our local machine
    # to our docker container. This means whenever you change some code or file in
    # the project, it will automatically get updated into the container. That means
    # you don't need to restart the docker to get the changes.
      - ./app:/app
    # ./app = 'app' directory in our local machine
    # /app =  'app' directory inside the docker image

    # Now we will use 'command' to run our application inside docker container.
    command: > # > = This symbol break the command to next line
      sh -c "python manage.py runserver 0.0.0.0:8000"
      # sh = We are going to run the command using shell
      # -c = Run following command
      

4. In the directory where docker-compose.yml file has, run following command in the windows powershell or command line
<br/><b>`$ docker-compose build`</b>
<br/>This command will build the image & give it the name of the folder `restapiwithdjangoudemy_app:latest`

5. Now we will create a django project, using the above docker configuration. Now create our django project by this command:
<br/><b>`$ sudo docker-compose run app sh -c "django-admin.py startproject app ."`</b>
<br/> app = Name of the service which you want to run. Currently we have only one service. Anything after this word will pass into this linux container.
<br/> sh  = Run a shell script inside linux container.
<br/> -c = This is the flag for passing command into linux container.
<br/> "" = Inside this we pass our command.
<br/> django-admin.py startproject app = This will create a new django project name `app`.
<br/> . = The project will be created in the current directory.

# Section-5:

### Video: 20. Create Travis-CI configuration file

The travis ci configuration file is the file, that tells travis ci what to do, everytime we push a change to our project.

You create a travis ci configuration file by simply creating file in the root of the project directory. You name the file as <b>`.travis.yml`</b>

In [ ]:
                                                                                          #.travis.yml
#In the first line we will tell travis CI what language we have used in this
#project. As we have created this project using python. So we will write.
language: python

# Next we will tell travis CI what version of python it should use to spin
# up the server to run automated test.
python:
 - "3.6"

#Next we are going to tell travis CI, what service we need to use.
services:
 - "docker"
# We are just going to need docker service for this project. And all of
#the sub services are gonna be contained within our docker-compose.yml file
#and our Dockerfile configuration.

#Now we can specify a before_script, which is a script that travis will run
#before it executes any of the automation command, that we are going to input
#after before_script.
before_script: pip install docker-compose
#Before it runs any thing we need to install docker compose.

#Next we specify the script.
script:
  #Here we are going to run our docker-compose command for running
  #our automated test.
  - docker-compose run app sh -c "python manage.py test && flake8"
  #We also will run our linting tool by adding "flake8" with the
  #above command.

Next we need to add this (flake8) linting tool in our project. For that we will add <b>`flake8>=3.6.0,<3.7.0`</b> in our requirement.txt file.

In [1]:
                                                                                     #requirements.txt
Django>=2.1.4,<2.2.0
djangorestframework>=3.9.0,<3.10.0
flake8>=3.6.0,<3.7.0

SyntaxError: invalid syntax (<ipython-input-1-e604a771ade5>, line 2)

Next we will add flake8 config file inside our django project. Name of the file will be <b>`.flake8`</b>

In [ ]:
                                                                                             #.flake8
[flake8]
exclude=
 migrations,
 __pycahce__,
 manage.py,
 settings.py

Afte adding `.flake8` config file in our project, lets rebuild our docker image by below command:
<br/><b>`$ sudo docker-compose build`</b>

Next commit & push our changes to the github.

# Section-6:

### Video: 21. Writing a simple unit test

Lets create a new file in our app directory name calc.py

In [ ]:
                                                                                            #calc.py
def add(x,y):
    """
    This function add two number together.
    """
    return x + y

Now lets say we want create an unit test for our add function. To do this we are going to create a new file name <b>`tests.py`</b> in the same directory.

Django unit test framework looks for any file /folder that begins with the word `test` and it basically use those file / folder as a test script, when you run django `run unit test` command. 

In [ ]:
                                                                                            #tests.py
from django.test import TestCase
# This above import will help us to test our django app.

#Next we will import function which we want to test.
from app.calc import add

#Now we will create a classs which will test our calc
#class.
class CalcTest(TestCase):
    #We are going to inherit this class from TestCase

    # Now we will create test function to test the add
    #function.

    def test_add_numbers(self):
        """
        Test that two numbers are added together.
        """
        self.assertEqual(add(3,8),11)

One thing to mention about this function name <b>`test_add_numbers`</b> is that, just like when django search for file / folder with the start name `test`, this test function should also start with the name `test`.

Now lets save this <b>`tests.py`</b> file & go to our terminal and run django unit test by this below command:
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test"`</b>

### Video: 22. Writing a unit test with TDD

We will also check python linting, So:
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test && flake8"`</b>

# Section 7: Configure Django custom user model

### Video: 23. Create core app

The first thing we're going to do is we're going to create a core app which will hold all of the central code that is important to the rest of the sub apps that we create in our system. So it's going to create anything that is shared between one or more apps. So things like the migration and the database I like to put this all in the core module just so it's all in one place and is very clear where the kind of central point of all these things is.

All right we're going to create our app from the terminal and we're going to use the django manage
command just like we did to create the project. So we're going to type:

<br/><b>`$ sudo docker-compose run app sh -c "python manage.py startapp core"`</b>

Let's clean up some of the files that we're not going to be using in this `core` folder. So we're not going to be using the `tests.py` file, So delete this file And let's remove the `views.py` because we're not going to need the views in the core. 

And then let's create a new folder called `tests` and inside this folder Let's create `__init__.py`.

### Video: 24. Add tests for custom user

First Let's go into the settings of our app and make sure that the core app that we created in the previous video is added to the installed apps list.

In [ ]:
#settings.py

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'core', # Here we have added our core app.
]

Once as added We can go into our `tests` folder and we can create a new file called `test_models.py`

In [ ]:
                                                                                      #test_models.py
from django.test import TestCase
from django.contrib.auth import get_user_model

class ModelTests(TestCase):

    def test_create_user_with_email_successfull(self):
        """
        Test that will test whether creating a new user with
        email and password is suceessful.
        """

        # Now create one fake email & password for testing the
        # user object
        email = "test_email@gmail.com"
        password = "TestPassword123"

        # Create fake user object using above emaill & pass
        user = get_user_model().objects.create_user(
            email = email
            password = password
        )

        # Test the fake user object, whether properly created
        # or not.
        self.assertEqual(user.email,email)
        self.assertTrue(user.check_password(password))

Now, we will run our test:
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test && flake8"`</b>

### Video: 25. Implement custom user model

The first thing we're going to do is we have to create a user model in our `models.py` file in the
core app and then we're going to update the `settings.py` file to set our custom auth user model.

In [1]:
                                                                                           #models.py
from django.db import models
from django.contrib.auth.models import AbstractBaseUser, \
    BaseUserManager, PermissionMixin

# Create a model manager class. As our model name is 'user',
# So, model manager class name will be 'UserModel'
class UserManager(BaseUserManager):

    def create_user(self, email, password=None, **extra_fields):
        """
        This function creates and saves a new user.

        email = User email address.
        password = User provided password. Default is none.
        **extra_fields = This will allow to add new field when
                    we create user. Such as user phone number
                    etc.

        return
        This function return a newly created user.
        """

        user = self.model(email=email,**extra_fields)
        user.set_password(password)
        user.save(using=self._db)

        # 'using=self._db' = This will help us to support
        # multiple database.

        return user

# Now we will create our model class

class User(AbstractBaseUser, PermissionMixin):
    """
    This is custom user model, that supports email base
    user authentication rather than name base user 
    authentication.
    """

    email = models.EmailField(max_length=255, unique=True)
    name = models.CharField(max_length=255)
    is_active = models.BooleanField(default=True)
    is_staff = models.BooleanField(default=False)

    objects = UserManager()

    USERNAME_FIELD = 'email'
    # In django by default USERNAME_FIELD = 'username', but
    # in this app we are stting it to be email.

In [ ]:
                                                                                         #settings.py

########################### At the bottom of the file add this.###################################

# Settings added by me
AUTH_USER_MODEL = 'core.User'
# Here 'core' is the name of the app
# And 'User' is the name of our model

Now go back to our terminal and make the migration.
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py makemigrations core"`</b>

Here,
core = It is the name of the app for which we are running this above migration command.

Now that we have complete our database migration successfully, we can run our test again.
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test && flake8"`</b>

### Video: 26. Normalize email addresses

In [ ]:
                                                                                       #test_models.py
from django.test import TestCase
from django.contrib.auth import get_user_model

class ModelTests(TestCase):

    def test_create_user_with_email_successfull(self):
        """
        Test that will test whether creating a new user with
        email and password is suceessful.
        """

        # Now create one fake email & password for testing the
        # user object
        email = "test_email@gmail.com"
        password = "TestPassword123"

        # Create fake user object using above emaill & pass
        user = get_user_model().objects.create_user(
            email = email,
            password = password
        )

        # Test the fake user object, whether properly created
        # or not.
        self.assertEqual(user.email,email)
        self.assertTrue(user.check_password(password))


    def test_new_user_email_normalized(self):
        """
        This test will check whether we have implemented
        email text normalized / convert all upper case letter
        in email to lowercase or not.
        """

        # Now we will create a dummy email with all upper case
        # after @ sign 
        email = "test_email@GMAIL.COM"
        pass = "Test1234"

        # Create fake user object using above emaill & pass
        user = get_user_model().objects.create_user(
            email = email,
            password = pass
        )

        # Now we will check whether our upper case email
        # has been converted into lower case.
        self.assertEqual(user.email, email.lower())


Now go back to our terminal & run only python test this time not flake8 test.
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test"`</b>

And we will see that our test failed. So to pass the test we will modify our `models.py` file.

In [ ]:
from django.db import models
from django.contrib.auth.models import AbstractBaseUser, \
    BaseUserManager, PermissionsMixin

# Create a model manager class. As our model name is 'user',
# So, model manager class name will be 'UserModel'
class UserManager(BaseUserManager):

    def create_user(self, email, password=None, **extra_fields):
        """
        This function creates and saves a new user.

        email = User email address.
        password = User provided password. Default is none.
        **extra_fields = This will allow to add new field when
                    we create user. Such as user phone number
                    etc.

        return
        This function return a newly created user.
        """

        user = self.model(
            email=self.normalize_email(email),
            **extra_fields
            )
        user.set_password(password)
        user.save(using=self._db)

        # 'using=self._db' = This will help us to support
        # multiple database.

        return user

# Now we will create our model class

class User(AbstractBaseUser, PermissionsMixin):
    """
    This is custom user model, that supports email base
    user authentication rather than name base user 
    authentication.
    """

    email = models.EmailField(max_length=255, unique=True)
    name = models.CharField(max_length=255)
    is_active = models.BooleanField(default=True)
    is_staff = models.BooleanField(default=False)

    objects = UserManager()

    USERNAME_FIELD = 'email'
    # In django by default USERNAME_FIELD = 'username', but
    # in this app we are stting it to be email.

### Video: 27. Add validation for email field

Next we're going to add validation to ensure that an e-mail field has actually been provided when create_user() function is called. So lets write new test in our `test_models.py` file.

In [ ]:
                                                                                    #test_models.py
from django.test import TestCase
from django.contrib.auth import get_user_model

class ModelTests(TestCase):

    def test_create_user_with_email_successfull(self):
        """
        Test that will test whether creating a new user with
        email and password is suceessful.
        """

        # Now create one fake email & password for testing the
        # user object
        email = "test_email@gmail.com"
        password = "TestPassword123"

        # Create fake user object using above emaill & pass
        user = get_user_model().objects.create_user(
            email = email,
            password = password
        )

        # Test the fake user object, whether properly created
        # or not.
        self.assertEqual(user.email,email)
        self.assertTrue(user.check_password(password))


    def test_new_user_email_normalized(self):
        """
        This test will check whether we have implemented
        email text normalized / convert all upper case letter
        in email to lowercase or not.
        """

        # Now we will create a dummy email with all upper case
        # after @ sign 
        email = "test_email@GMAIL.COM"
        password = "Test1234"

        # Create fake user object using above emaill & pass
        user = get_user_model().objects.create_user(
            email = email,
            password = password
        )

        # Now we will check whether our upper case email
        # has been converted into lower case.
        self.assertEqual(user.email, email.lower())


    def test_invalid_user_email_raise_value_error(self):
        """
        This test will ensure that if we try to create a new
        user without providing an emaill address, our function
        will raise a value error.
        """
        with self.assertRaises(ValueError):
            # anything that we run in here should 
            # raise the value error And if it doesn't 
            # raise the value error, then this test will fail.
            get_user_model().objects.create_user(None,'test123')

Now go back to our terminal & run the test .
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test"`</b>

And we will see that our test failed. So to pass the test we will modify our `models.py` file.

In [ ]:
                                                                                           #models.py
from django.db import models
from django.contrib.auth.models import AbstractBaseUser, \
    BaseUserManager, PermissionsMixin

# Create a model manager class. As our model name is 'user',
# So, model manager class name will be 'UserModel'
class UserManager(BaseUserManager):

    def create_user(self, email, password=None, **extra_fields):
        """
        This function creates and saves a new user.

        email = User email address.
        password = User provided password. Default is none.
        **extra_fields = This will allow to add new field when
                    we create user. Such as user phone number
                    etc.

        return
        This function return a newly created user.
        """
        if not email:
            raise ValueError('User must provide email address')

        user = self.model(
            email=self.normalize_email(email),
            **extra_fields
            )
        user.set_password(password)
        user.save(using=self._db)

        # 'using=self._db' = This will help us to support
        # multiple database.

        return user

# Now we will create our model class

class User(AbstractBaseUser, PermissionsMixin):
    """
    This is custom user model, that supports email base
    user authentication rather than name base user 
    authentication.
    """

    email = models.EmailField(max_length=255, unique=True)
    name = models.CharField(max_length=255)
    is_active = models.BooleanField(default=True)
    is_staff = models.BooleanField(default=False)

    objects = UserManager()

    USERNAME_FIELD = 'email'
    # In django by default USERNAME_FIELD = 'username', but
    # in this app we are stting it to be email.

### Video: 28. Add support for creating superusers

create_super_user() is a function used by the Django command line interface (cli), when we are creating new users using the command line.

In [ ]:
                                                                                     #test_models.py
from django.test import TestCase
from django.contrib.auth import get_user_model

class ModelTests(TestCase):

    def test_create_user_with_email_successfull(self):
        """
        Test that will test whether creating a new user with
        email and password is suceessful.
        """

        # Now create one fake email & password for testing the
        # user object
        email = "test_email@gmail.com"
        password = "TestPassword123"

        # Create fake user object using above emaill & pass
        user = get_user_model().objects.create_user(
            email = email,
            password = password
        )

        # Test the fake user object, whether properly created
        # or not.
        self.assertEqual(user.email,email)
        self.assertTrue(user.check_password(password))


    def test_new_user_email_normalized(self):
        """
        This test will check whether we have implemented
        email text normalized / convert all upper case letter
        in email to lowercase or not.
        """

        # Now we will create a dummy email with all upper case
        # after @ sign 
        email = "test_email@GMAIL.COM"
        password = "Test1234"

        # Create fake user object using above emaill & pass
        user = get_user_model().objects.create_user(
            email = email,
            password = password
        )

        # Now we will check whether our upper case email
        # has been converted into lower case.
        self.assertEqual(user.email, email.lower())


    def test_invalid_user_email_raise_value_error(self):
        """
        This test will ensure that if we try to create a new
        user without providing an emaill address, our function
        will raise a value error.
        """
        with self.assertRaises(ValueError):
            # anything that we run in here should 
            # raise the value error And if it doesn't 
            # raise the value error, then this test will fail.
            get_user_model().objects.create_user(None,'test123')

    def test_create_new_super_user(self):
        """
        This test will check whether a new user is 
        created and have the super user & staf tag 
        assigned to it.
        """
        user = get_user_model().objects.create_superuser(
            'test_123@gmail.com',
            'Test123'
        )

        # check whether new user is super user
        self.assertTrue(user.is_superuser)
        # check whether new user is staff
        self.assertTrue(user.is_staff)

Now go back to our terminal & run the test .
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test"`</b>

And we will see that our test failed. So to pass the test we will modify our `models.py` file.

In [ ]:
                                                                                        #models.py
from django.db import models
from django.contrib.auth.models import AbstractBaseUser, \
    BaseUserManager, PermissionsMixin

# Create a model manager class. As our model name is 'user',
# So, model manager class name will be 'UserModel'
class UserManager(BaseUserManager):

    def create_user(self, email, password=None, **extra_fields):
        """
        This function creates and saves a new user.

        email = User email address.
        password = User provided password. Default is none.
        **extra_fields = This will allow to add new field when
                    we create user. Such as user phone number
                    etc.

        return
        This function return a newly created user.
        """
        if not email:
            raise ValueError('User must provide email address')

        user = self.model(
            email=self.normalize_email(email),
            **extra_fields
            )
        user.set_password(password)
        user.save(using=self._db)

        # 'using=self._db' = This will help us to support
        # multiple database.

        return user

    
    def create_superuser(self, email, password):
        """
        Create a new super user & save it & return it
        """
        user = self.create_user(email, password)
        user.is_staff = True
        user.is_superuser = True
        user.save(using=self._db)

        return user

# Now we will create our model class

class User(AbstractBaseUser, PermissionsMixin):
    """
    This is custom user model, that supports email base
    user authentication rather than name base user 
    authentication.
    """

    email = models.EmailField(max_length=255, unique=True)
    name = models.CharField(max_length=255)
    is_active = models.BooleanField(default=True)
    is_staff = models.BooleanField(default=False)

    objects = UserManager()

    USERNAME_FIELD = 'email'
    # In django by default USERNAME_FIELD = 'username', but
    # in this app we are stting it to be email.

# Section 8: Setup Django admin

### Video: 29. Add tests for listing users in Django admin

Now we will create `test_admin.py` file in our test folder.

In [ ]:
                                                                                    #test_admin.py
from django.test import TestCase, Client
from django.contrib.auth import get_user_model
from django.urls import reverse

class AdminSiteTests(TestCase):

    def setUp(self):
        """
        This function is run before every test from this
        class is run. So some times there are some set up
        task, that need to be done before running every
        test from this class.
        """
        # First create a test client
        self.client=Client()
        # What this above line does is, it basically set
        # Client() to self, so that other function from
        # this class can access Client()

        # Create an admin, which will be used to logged
        # into the above client
        self.admin_user = get_user_model().objects.create_superuser(
            email='admin@gmail.com',
            password='admin123'
        )
        # Now make the admin login to the client
        self.client.force_login(self.admin_user)
        # This above line will help us to run our test smothly,
        # because now we don't have to manualy log the user in,
        # we can use this above force_login() helper function.

        self.user = get_user_model().objects.create_user(
            email='test@gmail.com',
            password='Test123',
            name='Full name of the test user'
        )

    def tes_user_listed(self):
        """
        Test that user's are listed on user page.
        """
        url = reverse('admin:core_user_changelist')
        res = self.client.get(url)

        self.assertContains(res, self.user.name)
        self.assertContains(res, self.user.email)

Now go back to our terminal & run the test .
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test"`</b>

### Video: 30. Modify Django admin to list our custom user model

Okay, now we can go ahead and customize our Django admin list in our custom user model. So lets go over to our `admin.py` file and modify this file like below.

In [ ]:
                                                                                         #admin.py
from django.contrib import admin
from django.contrib.auth.admin import UserAdmin As BaseUserAdmin
from core import models


class UserAdmin(BaseUserAdmin):
    """
    This our custom user admin.
    """
    ordering = ['id']
    list_display = ['email', 'name']


# Now we need to register our UserAdmin class
admin.site.register(models.User, UserAdmin)

Now go back to our terminal & run the test .
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test && flake8"`</b>

### Video: 31. Modify Django admin to support changing user model

So let's start by adding a test to `test_admin.py` and we're just going to test that the change page renders correctly.

In [ ]:
                                                                                    #test_admin.py
def test_user_change_page(self):
        """
        Test that the user edit page work.
        """
        url = reverse('admin:core_user_change', args=[self.user.id])
        # This reverse() function will create url like this:
        # /admin/core/user/1

        # Now we are going to do a http get on this above url.
        res = self.client.get(url)

        # Now we will test that this page rendered ok.
        self.assertEqual(res.status_code, 200)
        # 200 means rendered ok.


Now go back to our terminal & run the test .
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test && flake8"`</b><br/>
As usual our test failed, so head over to `admin.py` & modify.

In [ ]:
                                                                                        #admin.py
from django.contrib import admin
from django.contrib.auth.admin import UserAdmin as BaseUserAdmin
from django.utils.translation import gettext as _
from core import models


class UserAdmin(BaseUserAdmin):
    """
    This our custom user admin.
    """
    ordering = ['id']
    list_display = ['email', 'name']
    fieldsets = (
        (None, {'fields': ('email', 'password')}),
        (_('Personal Info'), {'fields': ('name',)}),
        (_('Permissions'),
            {'fields': ('is_active', 'is_staff', 'is_superuser')}),
        (_('Important Dates'), {'fields': ('last_login',)})
    )


# Now we need to register our UserAdmin class
admin.site.register(models.User, UserAdmin)


### Video: 32. Modify Django admin to support creating users

There's one last thing we need to change in our Django admin before we work with our custom user model and that is the Add page. This is the page for adding new users in the Django admin.

So let's go ahead and head over to our `test_admin.py` and lets add a new test that test add page rendered correctly.

In [ ]:
                                                                                    #test_admin.py
def test_create_user_page(self):
        """
        Test that will verify that create user page rendered
        correctly or not.
        """
        url = reverse('admin:core_user_add')
        # Now we are going to do a http get on this above url.
        res = self.client.get(url)

        # Now we will test that this page rendered ok.
        self.assertEqual(res.status_code, 200)
        # 200 means rendered ok.

Now go back to our terminal & run the test .
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test && flake8"`</b><br/>
As usual our test failed, so head over to `admin.py` & modify.

In [ ]:
                                                                                            #admin.py
from django.contrib import admin
from django.contrib.auth.admin import UserAdmin as BaseUserAdmin
from django.utils.translation import gettext as _
from core import models


class UserAdmin(BaseUserAdmin):
    """
    This our custom user admin.
    """
    ordering = ['id']
    list_display = ['email', 'name']
    fieldsets = (
        (None, {'fields': ('email', 'password')}),
        (_('Personal Info'), {'fields': ('name',)}),
        (_('Permissions'),
            {'fields': ('is_active', 'is_staff', 'is_superuser')}),
        (_('Important Dates'), {'fields': ('last_login',)})
    )
    add_fieldsets = (
        (None, {
            'classes': ('wide',),
            'fields': ('email', 'password1', 'password2')
        }),
    )


# Now we need to register our UserAdmin class
admin.site.register(models.User, UserAdmin)


# Section 9: Setting up database

### Video: 33. Add postgres to docker compose

In [ ]:
version: "3"
# docker-compose version number. 3 is the lattest version

# Now we will define services that make up our application
services:
  app: # This is the name of our services
    build:
      context: . # This will set the build to our current directory
    ports:
      - "8000:8000"
      # First 8000, means we are exposing port 8000 in our machine.
      # Second 8000, means we are exposing port 8000 in our docker image.
    volumes:
    # Volumes allows us to get the update (that we make to our project) into our docker
    # image in real time. So this section will maps a volume from our local machine
    # to our docker container. This means whenever you change some code or file in
    # the project, it will automatically get updated into the container. That means
    # you don't need to restart the docker to get the changes.
     - ./app:/app
    # ./app = 'app' directory in our local machine
    # /app =  'app' directory inside the docker image

    # Now we will use 'command' to run our application inside docker container.
    command: > # > = This symbol break the command to next line
      sh -c "python3 manage.py runserver 0.0.0.0:8000"
      # sh = We are going to run the command using shell
      # -c = Run following command
    environment:
      - DB_HOST=db
      # Here right side 'db' must match with the db service name.
      # As we have named our database service as db, that is why
      # we put 'DB_HOST=db'
      - DB_NAME=app
      # As in our db service's environment we have set
      # 'POSTGRES_DB=app', that is why above we have write
      # 'DB_NAME=app'
      - DB_USER=postgres
      # As in our db service's environment we have set
      # 'POSTGRES_USER=postgres'
      - DB_PASS=anypassword
      # As in our db service's environment we have set
      # 'POSTGRES_PASSWORD=anypassword'

    # When you run docker-compose you can set, different
    # services that depend on other services, by declaring
    # it on depends on section.
    depends_on:
      - db
      # We want our app service to  depend on our db
      # service. This means two things. 1) db service
      # will start before app sevice. 2) db service
      # will be able able via the network.
  db: # This is the name of our services
    image: postgres:10-alpine
    # Docker will locate 'postgres' image from docker hub
    # and it will pull down the version with a tag '10-alpine'
    environment:
      - POSTGRES_DB=app
      - POSTGRES_USER=postgres
      - POSTGRES_PASSWORD=anypassword

### Video: 34. Add postgres support to Dockerfile

We need to install the python package that is used by Django to communicate with postgres. So we will add this below dependency in our `requirements.txt` file.

In [ ]:
                                                            #requirements.txt
Django>=2.1.4,<2.2.0
djangorestframework>=3.9.0,<3.10.0
psycopg2>=2.7.5,<2.8.0 # This is the package used by Django to communicate with postgres
flake8>=3.6.0,<3.7.0

If we try to build this now, it would not work. Because there are some dependencies required in order to install this `psycopg2` package in any machine. So lets head over to our `Dockerfile` & add those dependency there.

In [ ]:
                                                                #Dockerfile
# Bring Base Image From The Docker Hub
FROM python:3.7-alpine

# Name Of The Code Maintainer & Email 
MAINTAINER Saifullah al Mujahid <mujahid7292@gmail.com>

# Let set python unbuffered environment variable.
ENV PYTHONUNBUFFERED 1
# What this environment variable does, is that it tell python
# to run in unbuffered mode. This is recommended when running 
# python inside docker container. The reason for this, is that 
# it does not allow python to buffer the output. So it reduce
# the error in running python inside docker image.

# Temp Workaround for DNS issue
RUN echo http://nl.alpinelinux.org/alpine/v3.9/main > /etc/apk/repositories; \
    echo http://nl.alpinelinux.org/alpine/v3.9/community >> /etc/apk/repositories
    
# We are going to store our projects dependencies in the
# ./requirements.txt file.
COPY ./requirements.txt  /requirements.txt
# This command will copy this file to inside the docker image
# in the following '/' directory.

# To install 'psycopg2' package from our 'requirements.txt'
# file, first we need to install 'postgresql-client'.
RUN apk add --update --no-cache postgresql-client
# This above run command will use the package manager
# that comes from python:3.7-alpine (which is apk)
# and we are going to add a package (by add command)
# and this '--update' command means update the registry
# before adding the package.
# --no-cache = means, don't store the registry index
# on our Dockerfile. The reason we do this, is because
# we really want to minimize number of extra files &
# packages that are included in our docker container.
# This is best practice, because it means that your
# docker container for your application has the smallest 
# footprint possible. And it also means, you don't include
# any extra dependencies on your system, which may cause
# sequrity vulnerabilities in your system.

# Now we are going to install some temporary packages
# that need to be installed on the system, while we run
# our requirements.txt file, then we can remove them
# after installing all of the package from requirements.txt
# file.
RUN apk add --update --no-cache --virtual .tmp-build-deps \
        gcc libc-dev linux-headers postgresql-dev
# --virtual = it help us to easilly remove those dependencies
# later. So here it is called '.tmp-build-deps' or temporary
# build dependencies.
# \ = means, go to next line.

# Now we will install all of the dependencies of our project.
RUN pip install -r /requirements.txt

# After installing all the dependecies / package from
# requirements.txt file, we will delete all our temporary
# build dependencies / packages
RUN apk del .tmp-build-deps

# We will create a directory(/app) within our docker image, that we can use
# to store our application source code.
RUN mkdir /app
# Create a directory/folder named 'app' inside our docker image

WORKDIR /app
# Switched to this 'app' directory as a default directory.
# Any application that we run inside our docker container, will start
# running from this location.

COPY ./app /app
# Copy all the source code from our local machine './app' folder to the directory
# inside docker container which is also '/app' folder.

# We will create an user who will run our application using docker.
RUN adduser -D mujahid7292
# -D = Create an user without admin privilage. That means this user will only be able 
# to run this docker image & will not get any root access.
# mujahid7292 = Name of the user.
USER mujahid7292
# Now docker will switched to this 'mujahid7292' user.

### Video: 35. Configure database in Django

Now we will modify our Django default database. As Django provide SqLite as the default database, we have to remove SqLite and add postgres database in our `settings.py` file.

In [ ]:
                                                                                         #settings.py
# Modify this portion of the code
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
    }
}

In [ ]:
                                                                                         #settings.py
# After modification
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql',
        'HOST': os.environ.get('DB_HOST'),
        'NAME': os.environ.get('DB_NAME'),
        'USER': os.environ.get('DB_USER'),
        'PASSWORD': os.environ.get('DB_PASS'),
    }
}

# Section 10: Waiting for postgres to start

### Video: 37. Add tests for wait_for_db command

In this video we're going to add a management command to the core app of our Django project. The management command is going to be a helper command, that allows us to wait for the database to be available before continuing and running other commands. We're going to use this command in our docker compose file when starting our Django app.

The reason that we need this command is because, I find that sometimes when using Postgres with Docker compose in a Django app, sometimes the Django app fails to start because of a database error.

It turns out that this is because, once the Postgres services started, there were a few some extra set up tasks that need to be done on the Postgres, before it is ready to accept connections.

So what this means is our Django app will try and connect to our database before the database is ready and therefore it will fail with an exception and you need to restart the app.

To improve the reliability of our project, We're going to add this helper command that we can put in front of all of the commands we run in Docker compose and that will ensure that a database is up and ready to accept connections, before we try and access the database. This will make our application a lot more reliable when running it locally as a development server and also if we ever deploy it as a production system.

So let's go ahead and create our `wait_for_db` command. We're going to start by creating the unit test for our command, which we will create in our core tests folder. So let's create a new file called `test_commands.py`

In [ ]:
                                                                                #test_commands.py
from unittest.mock import patch
# This above import is going to allow us to 
# mock the behaviour of the Django get_database() function.
# What this will do is, we can basically simulate the database 
# being available and not being available for when we test our
# command.

from django.core.management import call_command

from django.db.utils import OperationalError
# OperationalError is an error that Django throws when the database is 
# unavailable and we're going to use this error to simulator database as 
# being available or not.

from django.test import TestCase


class CommandTests(TestCase):
    """
    This class is resposible for doing all the test associated with
    command.
    """

    # The first function we're going to create is simply going to test,
    # what happens when we call our command and the database is already
    # available.
    def test_wait_for_db_ready(self):
        """
        Test waiting for db, when db is available.
        """
        # So to set up our test here we need to simulate the behaviour of
        # Django, When the database is available. Our management command is 
        # going to basically try and retrieve the database connection from
        # Django. And it's going to check if when we try and retrieve it,
        # it retrieved an operational error or not. So if it retrieves an
        # operational error then a database is not available. If an operational
        # error is not thrown then the database is available and the command
        # will continue.

        # So to set up our test we're going to override the behaviour of the
        # connection handler. And we're just going to make it return true and
        # not through any exception and therefore our call command or our
        # management command should just continue and allow us to continue
        # with the execution flow. So let's use the patch to mark the connection
        # handler to just return true every time it's called.
        with patch('django.db.utils.ConnectionHandler.__getitem__') as gi:
            gi.return_value = True
            call_command('wait_for_db')
            
            # Now we will assert that our db has been called once
            self.assertEqual(gi.call_count, 1)

    # The second function we're going to create is simply going to test,
    # That our wait_for_db() command will try 5 times to connect with 
    # the database & will fail to connect & in the 6th time our db
    # connection will be succesfull.
    @patch('time.sleep', return_value = True)
    def test_wait_for_db(self, ts): # ts = Time sleep
        """
        Test waiting for db.
        """
        with patch('django.db.utils.ConnectionHandler.__getitem__') as gi:
            gi.side_effect = [OperationalError] * 5 + [True]
            # Here 5 means, 5 times it will through operational error.
            # [True] means, on 6th time it will not through any
            # OperationalError.
            call_command('wait_for_db')

            self.assertEqual(gi.call_count, 6)

Now go back to our terminal & run the test .
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test && flake8"`</b><br/>
As usual our test failed, so head over & create `wait_for_db.py` file.

### Video: 38. Add wait_for_db command

Now we can go ahead and create our wait_for_db management command. To create the command, we're going to start by creating the directory in our core app that we were going to store our management commands.

So this is the Django convention and it's recommended on the Django website to put all of your commands in a directory called `management/commands`

So first we are going to create a folder called `management` inside our core app directory. Inside `management` folder we are going to add an `__init__.py` file and all this does is it make sure that this is picked up as a python module.

Within our `management` folder, we're going to add a new folder called `commands` and then within this we're also going to add an `__init__.py` file to it.

Inside this `commands` folder, we could add the name of the command that we want to create. Our command is going to be called `wait_for_db.py`, so we will create a file with this name.

In [ ]:
                                                                                    #wait_for_db.py
import time
from django.db import connections
# we can use 'connection' to test the database connection,
# whether it is available or not.
from django.db.utils import OperationalError
# This error is thrown by Django, when database is not
# available.
from django.core.management.base import BaseCommand
# And finally we're importing base command which is the class 
# that we need to build on in order to create our custom command class.

class Command(BaseCommand):
    """
    This is a Django command, to pause execution until database is
    available.
    """

    def handle(self, *args, **options):
        self.stdout.write('Waiting for database........')
        db_conn = None
        while not db_conn:
            # Means while db_conn = None, execution will enter
            # this loop.
            try:
                db_conn = connections['default']
            except OperationalError:
                self.stdout.write('Database unavailable, waiting for 1 sec.')
                time.sleep(1) # Our code will pause for 1 sec
        self.stdout.write(self.style.SUCCESS('Database is available!!!'))


### Video: 39. Make docker compose wait for db

Okay, Now we have our wait_for_db command,  we can go ahead and configure Docker compose to use this command before it starts our Django app.

In [ ]:
                                                                                #docker-compose.yml
version: "3"
# docker-compose version number. 3 is the lattest version

# Now we will define services that make up our application
services:
  app: # This is the name of our services
    build:
      context: . # This will set the build to our current directory
    ports:
      - "8000:8000"
      # First 8000, means we are exposing port 8000 in our machine.
      # Second 8000, means we are exposing port 8000 in our docker image.
    volumes:
    # Volumes allows us to get the update (that we make to our project) into our docker
    # image in real time. So this section will maps a volume from our local machine
    # to our docker container. This means whenever you change some code or file in
    # the project, it will automatically get updated into the container. That means
    # you don't need to restart the docker to get the changes.
     - ./app:/app
    # ./app = 'app' directory in our local machine
    # /app =  'app' directory inside the docker image

    # Now we will use 'command' to run our application inside docker container.
    command: > # > = This symbol break the command to next line
      sh -c "python3 manage.py wait_for_db && 
             python3 manage.py migrate && 
             python3 manage.py runserver 0.0.0.0:8000"
      # sh = We are going to run the command using shell
      # -c = Run following command
    environment:
      - DB_HOST=db
      # Here right side 'db' must match with the db service name.
      # As we have named our database service as db, that is why
      # we put 'DB_HOST=db'
      - DB_NAME=app
      # As in our db service's environment we have set
      # 'POSTGRES_DB=app', that is why above we have write
      # 'DB_NAME=app'
      - DB_USER=postgres
      # As in our db service's environment we have set
      # 'POSTGRES_USER=postgres'
      - DB_PASS=anypassword
      # As in our db service's environment we have set
      # 'POSTGRES_PASSWORD=anypassword'

    # When you run docker-compose you can set, different
    # services that depend on other services, by declaring
    # it on depends on section.
    depends_on:
      - db
      # We want our app service to  depend on our db
      # service. This means two things. 1) db service
      # will start before app sevice. 2) db service
      # will be able able via the network.
  db: # This is the name of our services
    image: postgres:10-alpine
    # Docker will locate 'postgres' image from docker hub
    # and it will pull down the version with a tag '10-alpine'
    environment:
      - POSTGRES_DB=app
      - POSTGRES_USER=postgres
      - POSTGRES_PASSWORD=anypassword

Now we will go to our terminal and run:
<br/><b>`$ sudo docker-compose up`</b>

### Video: 40. Test in browser

First start our docker compose server by below command:
<br/><b>`$ sudo docker-compose up`</b>

Now open google chrome browser & go to below address.
<br/><b>`http://127.0.0.1:8000/`</b><br/>
This above url will take us Django landing page and this will confirm us
that Django has been successfully installed in our system.

Once we are in this page, we can add `/admin/` with our above url. So, full url:
<br/><b>`http://127.0.0.1:8000/admin/`</b><br/>

We will see a page, where Django need email & password. As we did not create any super user yet, so we don't have any email & password to log in to the Django admin page. So lets create a super user through the command line. So open new command line and type below command to create super user.
<br/><b>`$ sudo docker-compose run app sh -c "python3 manage.py createsuperuser"`</b>

Then log in with newly created super user.

# Section Section 11: Create user management endpoints

### Video: 42. Create users app

In this section we're going to create our manage user endpoints.These endpoints are going to allow us, to create users, to update users, to change a user's password and to create user authentication tokens which can be used to authenticate requests to the other APIs in our project.

We're going to start by creating a user's app in our Django project. So we will run below command to create our user app.

<br/><b>`$ sudo docker-compose run --rm app sh -c "python manage.py startapp user"`</b><br/>

`--rm` = what this does is it removes the container, after it's ran the command. Now you can include this optionally on any commands that you just want to run once and you don't want the docker container to linger on the system after it's ran.So basically if we add the rm command it should remove the container and just keep the system a little cleaner so it doesn't fill up.

we're just going to do some cleanup here so we can remove the `migrations` folder because we're going to keep all of them within the core app and we can also remove the `admin.py` because we're also going to keep them in the core app and let's also remove the `models.py` here.Because again they're in the core app and finally let's remove these `tests.py`, because we're going to create a new sub folder for `tests`.

Finally what we're going to do is we're going to open up our app settings, that's under app / settings and we're going to add a few things here to the installed apps.

In [ ]:
                                                                                    #app/settings.py
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'rest_framework',
    'rest_framework.authtoken',
    'core',
    'user',
]

### Video: 43. Add tests for create user API

So we're going to start by adding some unit tests to test, creating users and different scenarios when we give different post requests to create user.

In [ ]:
                                                                                    #test_user_api.py
from django.test import TestCase
from django.contrib.auth import get_user_model
from django.urls import reverse

# Now we will import some rest_framework test helper tools
from rest_framework.test import APIClient
# This just is a test client that we can use to make requests
# to our API and then check what the response is.
from rest_framework import status
# This is a module that contains some status codes that
# we can see in basically human readable form. So instead
# of just typing 200, it's HTTP 200 ok. It just makes the
# tests a little bit easier to read and understand.

CREATE_USER_URL = reverse('user:create')

# Then we're going to add a helper function that we can
# use to create some example users for our tests.
# So anything that you do multiple times in different tests,
# I like to create a helper function so instead of creating
# the user for each test individually you can just call the
# helper function and it just makes it a little bit easier
# to create users that you're testing with.
def create_user(**params):
    return get_user_model().objects.create_user(**params)

# So a public API is one that is unauthenticated so that is
# just anyone from the internet can make a request. An example
# of this would be the create user, because when you typically
# create a user on a system, usually you're creating a user
# because you haven't got authentication set up already.
class PublicUserAPITest(TestCase):
    """
    Test publicly available user api.
    """

    def setUp(self):
        """
        This function is run before every test from this
        class is run. So some times there are some set up
        task, that need to be done before running every
        test from this class.
        """
        # First create a test client
        self.client = APIClient()
        # What this above line does is, it basically set
        # APIClient() to self, so that other function from
        # this class can access APIClient()

    # We're going to create a test that validates the user
    # is created successfully.
    def test_create_valid_user_success(self):
        """
        Test creating user with valid payload is successfull.
        """
        # First create a valid payload
        payload = {
            'email':'test@gmail.com',
            'password':'testpass123',
            'name':'Test user name'
        }

        # Now we will make our request to CREATE_USER_URL using
        # this avobe valid payload.
        res = self.client.post(CREATE_USER_URL, payload)
        # Here, our res object has user & http response status.

        # Now we will check our http response.
        self.assertEqual(res.status_code, status.HTTP_201_CREATED)

        # Next we're going to test that the user object is actually
        # created. So we will retrieve created user from django db.
        user = get_user_model().objects.get(**res.data)
        
        # Now we will check the retrieved user password, with our
        # payload password. If match then, user creation is successfull.
        self.assertTrue(user.check_password(payload['password']))

        # finally we want to check that the password is not returned
        # as part of this above object 'user'
        self.assertNotIn('password', res.data)

    # Next we're going to test what happens if they try and create
    # a user but the user already exists. So we're trying to create
    # duplicate user here.
    def test_user_exist(self):
        """
        Test creating a user that already exist fail.
        """
        # Generate a valid payload
        payload = {
            'email':'test@gmail.com',
            'password':'testpass123',
            'name':'Test user name'
        }
        # Create new user using create_user() function.
        create_user(**payload)

        # Now we will, try to create the same user by http post method.
        res = self.client.post(CREATE_USER_URL, payload)
        # Here, our res object has user & http response status.

        # We expect to see here is a HTTP 400 bad request because
        # it's a bad request because the user already exists.
        self.assertEqual(res.status_code, status.HTTP_400_BAD_REQUEST)
    
    # Now we are going to create a test which will check whether
    # the password is too short.
    def test_password_too_short(self):
        """
        Test password must be more than 5 characters
        """
        # Generate a payload with very short password
        payload = {
            'email':'test@gmail.com',
            'password':'pw',
            'name':'Test user name'
        }

        # Now we will make our request to CREATE_USER_URL using
        # this avobe payload.
        res = self.client.post(CREATE_USER_URL, payload)
        # Here, our res object has user & http response status.

        # So we want to first make sure that it returns a HTTP bad
        # request.
        self.assertEqual(res.status_code, status.HTTP_400_BAD_REQUEST)

        # Now lets check that the user was never created.
        user_exist = get_user_model().objects.filter(
            email = payload['email']
        ).exists()
        # What this above code will do is, if the user exists it will
        # return true otherwise it will return false.
        self.assertFalse(user_exist)

Now go back to our terminal & run the test .
<br/><b>`$ sudo docker-compose run --rm app sh -c "python manage.py test && flake8"`</b><br/>
As usual our test failed, so head over & create `test_user_api.py` file.

### Video: 44. Add create user API

What we're going to do is we're going to create a serializer for our create user request and then we're going to create a view which will handle the request and then we're going to wire this up to a URL which will allow us to access the API and also make our tests pass.

Let's start by creating a `serializers.py` file in the user app.

In [ ]:
                                                                             #user/serializers.py
from django.contrib.auth import get_user_model
from rest_framework import serializers

class UserSerializer(serializers.ModelSerializer):
    """
    Serializer for the user object.
    """
    class Meta:
        model = get_user_model()
        # This above line will give us the model, which we will
        # serialize in this class.

        # Next you can specify the fields that you want to include
        # in serializer so these are the fields that are going to
        # be converted to and from json when we make our HTTP POST
        # request. So those basically are the fields that we want
        # to make accessible in the API either to read or write.
        fields = ('email', 'password', 'name')
        # These are the three fields that we're going to accept
        # when we create users.

        # So finally we're going to add something called extra
        # keyword args and what this does is it allows us to configure
        # a few extra settings in our model serializer and what we're
        # going to use this for is to ensure that the password is write
        # only and that the minimum required length is 5 characters.
        extra_kwargs = {
            'password':{
                'write_only':True,
                'min_length':5
                }
            }
    
    def create(self, validated_data):
        """
        Create a new user with an encrypted password and
        return it.
        """
        return get_user_model().objects.create_user(**validated_data)

let's update our `views.py` and let's add a view for managing our create user API.

In [ ]:
                                                                                #user/views.py
from rest_framework import generics
from user.serializers import UserSerializer

# We're going to create a new view and that view are going to inherit
# from create API view, that comes with the Django rest framework.
# So this is a view that's pre-made for us that allows us to easily
# make a API that creates an object in a database using the serialize
# that we're going to provide.
class CreateUserView(generics.CreateAPIView):
    """
    Create a new user in the system.
    """
    serializer_class = UserSerializer

Finally before we can actually access our API we need to add a URL and wire the URL up to our view.
So we're going to create a new file in our user app called `urls.py`

In [ ]:
                                                                                    #user/urls.py
from django.urls import path
from user import views

# Then we can define our app name and the app name is set to
# help identify which app we're creating the URL from.
app_name = 'user'

urlpatterns = [
    path('create/', views.CreateUserView.as_view(), name = 'create'),
]

Now we have to add our `user/urls.py` url into our main `app/urls.py`

In [ ]:
                                                                                    #app/urls.py
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('api/user/', include('user/urls')), # This line added in this video
]

Now go back to our terminal & run the test .
<br/><b>`$ sudo docker-compose run --rm app sh -c "python manage.py test && flake8"`</b><br/>

### Video: 45. Add tests for creating a new token

Okay so the next thing we're going to add to our users API is the create token endpoint. This is going to be an endpoint that you can make a HTTP POST request and you can generate a temporary auth token that you can then use to authenticate future requests with the API.

With our API we're going to be using token authentication. So the way that you log in, is you use this API to generate a token and then you provide that token as the authentication header for future
requests which you want to authenticate. The benefit of this is you don't need to send the user's username and password with every single request that you make. You just need to send it once to create the token and then you can use that token for future requests and if you ever want to revoke the token you can do that in the database.

We're going to start by creating 4 unit tests.

1) First unit test will test that the token is created okay.

2) Second unit test will check, what happens if we provide invalid credentials.

3) Third unit test will check, if you're trying to authenticate against a non-existent user.

4) Fourth unit test will check, if you provide a request that doesn't include a password.

So let's get started and add our tests. We're going to add them to the `test_user_api.py` file in our
users tests directory.

In [ ]:
                                                                                #test_user_api.py
from django.test import TestCase
from django.contrib.auth import get_user_model
from django.urls import reverse

# Now we will import some rest_framework test helper tools
from rest_framework.test import APIClient
# This just is a test client that we can use to make requests
# to our API and then check what the response is.
from rest_framework import status
# This is a module that contains some status codes that
# we can see in basically human readable form. So instead
# of just typing 200, it's HTTP 200 ok. It just makes the
# tests a little bit easier to read and understand.

CREATE_USER_URL = reverse('user:create')
TOKEN_URL = reverse('user:token')

# Then we're going to add a helper function that we can
# use to create some example users for our tests.
# So anything that you do multiple times in different tests,
# I like to create a helper function so instead of creating
# the user for each test individually you can just call the
# helper function and it just makes it a little bit easier
# to create users that you're testing with.


def create_user(**params):
    return get_user_model().objects.create_user(**params)

# So a public API is one that is unauthenticated so that is
# just anyone from the internet can make a request. An example
# of this would be the create user, because when you typically
# create a user on a system, usually you're creating a user
# because you haven't got authentication set up already.


class PublicUserAPITest(TestCase):
    """
    Test publicly available user api.
    """

    def setUp(self):
        """
        This function is run before every test from this
        class is run. So some times there are some set up
        task, that need to be done before running every
        test from this class.
        """
        # First create a test client
        self.client = APIClient()
        # What this above line does is, it basically set
        # APIClient() to self, so that other function from
        # this class can access APIClient()

    # We're going to create a test that validates the user
    # is created successfully.
    def test_create_valid_user_success(self):
        """
        Test creating user with valid payload is successfull.
        """
        # First create a valid payload
        payload = {
            'email': 'test@gmail.com',
            'password': 'testpass123',
            'name': 'Test user name'
        }

        # Now we will make our request to CREATE_USER_URL using
        # this avobe valid payload.
        res = self.client.post(CREATE_USER_URL, payload)
        # Here, our res object has user & http response status.

        # Now we will check our http response.
        self.assertEqual(res.status_code, status.HTTP_201_CREATED)

        # Next we're going to test that the user object is actually
        # created. So we will retrieve created user from django db.
        user = get_user_model().objects.get(**res.data)

        # Now we will check the retrieved user password, with our
        # payload password. If match then, user creation is successfull.
        self.assertTrue(user.check_password(payload['password']))

        # finally we want to check that the password is not returned
        # as part of this above object 'user'
        self.assertNotIn('password', res.data)

    # Next we're going to test what happens if they try and create
    # a user but the user already exists. So we're trying to create
    # duplicate user here.
    def test_user_exist(self):
        """
        Test creating a user that already exist fail.
        """
        # Generate a valid payload
        payload = {
            'email': 'test@gmail.com',
            'password': 'testpass123',
            'name': 'Test user name'
        }
        # Create new user using create_user() function.
        create_user(**payload)

        # Now we will, try to create the same user by http post method.
        res = self.client.post(CREATE_USER_URL, payload)
        # Here, our res object has user & http response status.

        # We expect to see here is a HTTP 400 bad request because
        # it's a bad request because the user already exists.
        self.assertEqual(res.status_code, status.HTTP_400_BAD_REQUEST)

    # Now we are going to create a test which will check whether
    # the password is too short.
    def test_password_too_short(self):
        """
        Test password must be more than 5 characters
        """
        # Generate a payload with very short password
        payload = {
            'email': 'test@gmail.com',
            'password': 'pw',
            'name': 'Test user name'
        }

        # Now we will make our request to CREATE_USER_URL using
        # this avobe payload.
        res = self.client.post(CREATE_USER_URL, payload)
        # Here, our res object has user & http response status.

        # So we want to first make sure that it returns a HTTP bad
        # request.
        self.assertEqual(res.status_code, status.HTTP_400_BAD_REQUEST)

        # Now lets check that the user was never created.
        user_exist = get_user_model().objects.filter(
            email=payload['email']
        ).exists()
        # What this above code will do is, if the user exists it will
        # return true otherwise it will return false.
        self.assertFalse(user_exist)

    def test_create_token_for_user(self):
        """
        Test the a token is created for the user.
        """
        # First we will create a user.
        payload = {
            'email': 'test@gmail.com',
            'password': 'testpass123',
            'name': 'Test user name'
        }
        # Create new user using create_user() function.
        create_user(**payload)

        # Now as our user has been created in our system, we will
        # check whether our system give us token for that user or
        # not. Now we will make our request to TOKEN_URL using
        # this avobe payload.
        res = self.client.post(payload, TOKEN_URL)
        # Here, our res object has token & http response status.

        # First we will check whether our system has given this user
        # authentication token or not.
        self.assertIn('token', res.data)
        # So this assertion will check whether there is a key called
        # 'token' in the res.data that we get back.

        # Next we will assert that response is HTTP 200
        self.assertEqual(res.status_code, status.HTTP_200_OK)

    # Now we're going to test what happens if we provide invalid
    # credentials.
    def test_create_token_invalid_credential(self):
        """
        Test that token is not created, if invalid credential is
        given.
        """
        # First we will create an user in our system.
        create_user(email='mujahid7292@gmail.com', password='123456')

        # Now we will create an invalid sign in attempt, means
        # user will provide valid email, but wrong password.
        payload = {
            'email': 'mujahid7292@gmail.com',
            'password': 'wrongPassword'
        }

        # Now we will try this invalid sign in
        res = self.client.post(TOKEN_URL, payload)
        # This res object generally contain the authentication
        # token & response status.

        # Now we will assert that, this res object does not have
        # token in it.
        self.assertNotIn('token', res.data)

        # Next we will assert that response is HTTP_400_BAD_REQUEST
        self.assertEqual(res.status_code, status.HTTP_400_BAD_REQUEST)

    # In the next unit test we will check, if you're trying to authenticate
    # a non-existent user, what happen.
    def test_create_token_no_user(self):
        """
        Test that token is not created, if user does not exist.
        """
        # Now create our valid payload
        payload = {
            'email': 'mujahid7292@gmail.com',
            'password': '123456'
        }

        # Now we will try to authenticate with above user name
        # and password in our system, without creating user
        # with this exact email & password.

        # Now we will try this invalid sign in
        res = self.client.post(TOKEN_URL, payload)
        # This res object generally contain the authentication
        # token & response status.

        # Now we will assert that, this res object does not have
        # token in it.
        self.assertNotIn('token', res.data)

        # Next we will assert that response is HTTP_400_BAD_REQUEST
        self.assertEqual(res.status_code, status.HTTP_400_BAD_REQUEST)

    # this unit test will check, if you provide a request that doesn't
    # include a password, token is not returned.
    def test_create_token_missing_field(self):
        """
        Test that to authenticate with the server, email and
        password is required.
        """
        res = self.client.post(TOKEN_URL, {'email': 'one', 'password': ''})
        # This res object generally contain the authentication
        # token & response status.

        # Now we will assert that, this res object does not have
        # token in it.
        self.assertNotIn('token', res.data)

        # Next we will assert that response is HTTP_400_BAD_REQUEST
        self.assertEqual(res.status_code, status.HTTP_400_BAD_REQUEST)



Now go back to our terminal & run the test .
<br/><b>`$ sudo docker-compose run --rm app sh -c "python manage.py test && flake8"`</b><br/>

### Video: 46. Add create token API

Okay so next we can go ahead and implement our create token API to make our unit tests pass again.
So let's head over to our `user/serializers.py` where we're going to create our AuthTokenSerializer.

In [ ]:
                                                                            #user/serializers.py
from django.contrib.auth import get_user_model, authenticate
# authenticate = it's a Django helper command for working with
# the Django authentication system. So you simply pass in the
# username and password and you can authenticate a request.
from django.utils.translation import ugettext_lazy as _
from rest_framework import serializers


class UserSerializer(serializers.ModelSerializer):
    """
    Serializer for the user object.
    """
    class Meta:
        model = get_user_model()
        # This above line will give us the model, which we will
        # serialize in this class.

        # Next you can specify the fields that you want to include
        # in serializer so these are the fields that are going to
        # be converted to and from json when we make our HTTP POST
        # request. So those basically are the fields that we want
        # to make accessible in the API either to read or write.
        fields = ('email', 'password', 'name')
        # These are the three fields that we're going to accept
        # when we create users.

        # So finally we're going to add something called extra
        # keyword args and what this does is it allows us to configure
        # a few extra settings in our model serializer and what we're
        # going to use this for is to ensure that the password is write
        # only and that the minimum required length is 5 characters.
        extra_kwargs = {
            'password': {
                'write_only': True,
                'min_length': 5
                }
            }

    def create(self, validated_data):
        """
        Create a new user with an encrypted password and
        return it.
        """
        return get_user_model().objects.create_user(**validated_data)


    class AuthTokenSerializer(serializers.Serializer):
        """
        Serializer for the user authentication object.
        """
        email = serializers.CharField()
        password = serializers.CharField(
            style={'input_type': 'password'},
            trim_whitespace=False
        )

        # This validate function is called when we validate our serializer.
        # This validation function will check whether inputs are all correct,
        # like whether email is CharField etc And as part of the validation
        # function we are also going to validate that the authentication
        # credentials are correct.
        def validate(self, attrs):
            """
            Validate and authenticate the user.
            """
            # attrs = This is a dictionary, which contain our passed in
            # email and password. So this attrs parameter here is basically
            # just every field that makes up our serializer. So any field
            # that makes up a serializer, it will get passed into the
            # validate function here as this dictionary and then we can
            # retrieve the fields via this attributes and we can
            # then validate whether we want to pass this validation or we want
            # to fail the validation. Now we will rtrieve those email and
            # password.
            email = attrs.get('email')
            password = attrs.get('password')

            # Now we will authenticate our request.
            user = authenticate(
                request=self.context.get('request'),
                username=email,
                password=password
            )

            # If our above authentication fail, then
            if not user:
                # Means our authentication failed.
                msg = _('Unable to authenticate with provided credentials')
                raise serializers.ValidationError(
                    msg, 
                    code='authentication'
                )
            # Now we will return our 'attrs' dictionary by
            # inserting our authenticated user.
            attrs['user'] = user
            return attrs

In [ ]:
                                                                                    #user/views.py
from rest_framework import generics
from rest_framework.authtoken.views import ObtainAuthToken
from rest_framework.settings import api_settings
from user.serializers import UserSerializer, AuthTokenSerializer

# We're going to create a new view and that view are going to inherit
# from create API view, that comes with the Django rest framework.
# So this is a view that's pre-made for us that allows us to easily
# make a API that creates an object in a database using the serialize
# that we're going to provide.


class CreateUserView(generics.CreateAPIView):
    """
    Create a new user in the system.
    """
    serializer_class = UserSerializer

class CreateAuthToken(ObtainAuthToken):
    """
    Create a new auth token for user.
    """
    serializer_class = AuthTokenSerializer
    renderer_classes  = api_settings.DEFAULT_RENDERER_CLASSES

In [ ]:
                                                                                    #user/urls.py
from django.urls import path
from user import views

# Then we can define our app name and the app name is set to
# help identify which app we're creating the URL from.
app_name = 'user'

urlpatterns = [
    path('create/', views.CreateUserView.as_view(), name='create'),
    path('token/', views.CreateAuthToken.as_view(), name='token'),
]


Now go back to our terminal & run the test .
<br/><b>`$ sudo docker-compose run --rm app sh -c "python manage.py test && flake8"`</b><br/>

### Video: 47. Add tests for manage user endpoint

Next we're going to add our manage user endpoint. The manage user endpoint will allow the authenticated user to update their own profile this includes; changing their name, changing their password, and also viewing their user object so they can see what the values are currently set to.

So let's go ahead and add our manage user endpoint. We're going to start by adding tests in our `user/tests/test_user_api.py` file.

In [ ]:
                                                                        #user/tests/test_user_api.py
from django.test import TestCase
from django.contrib.auth import get_user_model
from django.urls import reverse

# Now we will import some rest_framework test helper tools
from rest_framework.test import APIClient
# This just is a test client that we can use to make requests
# to our API and then check what the response is.
from rest_framework import status
# This is a module that contains some status codes that
# we can see in basically human readable form. So instead
# of just typing 200, it's HTTP 200 ok. It just makes the
# tests a little bit easier to read and understand.

CREATE_USER_URL = reverse('user:create')
TOKEN_URL = reverse('user:token')
ME_URL = reverse('user:me') 
# ME_URL = This is update user end point.

# Then we're going to add a helper function that we can
# use to create some example users for our tests.
# So anything that you do multiple times in different tests,
# I like to create a helper function so instead of creating
# the user for each test individually you can just call the
# helper function and it just makes it a little bit easier
# to create users that you're testing with.


def create_user(**params):
    return get_user_model().objects.create_user(**params)

# So a public API is one that is unauthenticated so that is
# just anyone from the internet can make a request. An example
# of this would be the create user, because when you typically
# create a user on a system, usually you're creating a user
# because you haven't got authentication set up already.


class PublicUserAPITest(TestCase):
    """
    Test publicly available user api.
    """

    def setUp(self):
        """
        This function is run before every test from this
        class is run. So some times there are some set up
        task, that need to be done before running every
        test from this class.
        """
        # First create a test client
        self.client = APIClient()
        # What this above line does is, it basically set
        # APIClient() to self, so that other function from
        # this class can access APIClient()

    # We're going to create a test that validates the user
    # is created successfully.
    def test_create_valid_user_success(self):
        """
        Test creating user with valid payload is successfull.
        """
        # First create a valid payload
        payload = {
            'email': 'test@gmail.com',
            'password': 'testpass123',
            'name': 'Test user name'
        }

        # Now we will make our request to CREATE_USER_URL using
        # this avobe valid payload.
        res = self.client.post(CREATE_USER_URL, payload)
        # Here, our res object has user & http response status.

        # Now we will check our http response.
        self.assertEqual(res.status_code, status.HTTP_201_CREATED)

        # Next we're going to test that the user object is actually
        # created. So we will retrieve created user from django db.
        user = get_user_model().objects.get(**res.data)

        # Now we will check the retrieved user password, with our
        # payload password. If match then, user creation is successfull.
        self.assertTrue(user.check_password(payload['password']))

        # finally we want to check that the password is not returned
        # as part of this above object 'user'
        self.assertNotIn('password', res.data)

    # Next we're going to test what happens if they try and create
    # a user but the user already exists. So we're trying to create
    # duplicate user here.
    def test_user_exist(self):
        """
        Test creating a user that already exist fail.
        """
        # Generate a valid payload
        payload = {
            'email': 'test@gmail.com',
            'password': 'testpass123',
            'name': 'Test user name'
        }
        # Create new user using create_user() function.
        create_user(**payload)

        # Now we will, try to create the same user by http post method.
        res = self.client.post(CREATE_USER_URL, payload)
        # Here, our res object has user & http response status.

        # We expect to see here is a HTTP 400 bad request because
        # it's a bad request because the user already exists.
        self.assertEqual(res.status_code, status.HTTP_400_BAD_REQUEST)

    # Now we are going to create a test which will check whether
    # the password is too short.
    def test_password_too_short(self):
        """
        Test password must be more than 5 characters
        """
        # Generate a payload with very short password
        payload = {
            'email': 'test@gmail.com',
            'password': 'pw',
            'name': 'Test user name'
        }

        # Now we will make our request to CREATE_USER_URL using
        # this avobe payload.
        res = self.client.post(CREATE_USER_URL, payload)
        # Here, our res object has user & http response status.

        # So we want to first make sure that it returns a HTTP bad
        # request.
        self.assertEqual(res.status_code, status.HTTP_400_BAD_REQUEST)

        # Now lets check that the user was never created.
        user_exist = get_user_model().objects.filter(
            email=payload['email']
        ).exists()
        # What this above code will do is, if the user exists it will
        # return true otherwise it will return false.
        self.assertFalse(user_exist)

    def test_create_token_for_user(self):
        """
        Test the a token is created for the user.
        """
        # First we will create a user.
        payload = {
            'email': 'test@gmail.com',
            'password': 'testpass123',
            'name': 'Test user name'
        }
        # Create new user using create_user() function.
        create_user(**payload)

        # Now as our user has been created in our system, we will
        # check whether our system give us token for that user or
        # not. Now we will make our request to TOKEN_URL using
        # this avobe payload.
        res = self.client.post(TOKEN_URL, payload)
        # Here, our res object has token & http response status.

        # First we will check whether our system has given this user
        # authentication token or not.
        self.assertIn('token', res.data)
        # So this assertion will check whether there is a key called
        # 'token' in the res.data that we get back.

        # Next we will assert that response is HTTP 200
        self.assertEqual(res.status_code, status.HTTP_200_OK)

    # Now we're going to test what happens if we provide invalid
    # credentials.
    def test_create_token_invalid_credential(self):
        """
        Test that token is not created, if invalid credential is
        given.
        """
        # First we will create an user in our system.
        create_user(email='mujahid7292@gmail.com', password='123456')

        # Now we will create an invalid sign in attempt, means
        # user will provide valid email, but wrong password.
        payload = {
            'email': 'mujahid7292@gmail.com',
            'password': 'wrongPassword'
        }

        # Now we will try this invalid sign in
        res = self.client.post(TOKEN_URL, payload)
        # This res object generally contain the authentication
        # token & response status.

        # Now we will assert that, this res object does not have
        # token in it.
        self.assertNotIn('token', res.data)

        # Next we will assert that response is HTTP_400_BAD_REQUEST
        self.assertEqual(res.status_code, status.HTTP_400_BAD_REQUEST)

    # In the next unit test we will check, if you're trying to authenticate
    # a non-existent user, what happen.
    def test_create_token_no_user(self):
        """
        Test that token is not created, if user does not exist.
        """
        # Now create our valid payload
        payload = {
            'email': 'mujahid7292@gmail.com',
            'password': '123456'
        }

        # Now we will try to authenticate with above user name
        # and password in our system, without creating user
        # with this exact email & password.

        # Now we will try this invalid sign in
        res = self.client.post(TOKEN_URL, payload)
        # This res object generally contain the authentication
        # token & response status.

        # Now we will assert that, this res object does not have
        # token in it.
        self.assertNotIn('token', res.data)

        # Next we will assert that response is HTTP_400_BAD_REQUEST
        self.assertEqual(res.status_code, status.HTTP_400_BAD_REQUEST)

    # this unit test will check, if you provide a request that doesn't
    # include a password, token is not returned.
    def test_create_token_missing_field(self):
        """
        Test that to authenticate with the server, email and
        password is required.
        """
        res = self.client.post(TOKEN_URL, {'email': 'one', 'password': ''})
        # This res object generally contain the authentication
        # token & response status.

        # Now we will assert that, this res object does not have
        # token in it.
        self.assertNotIn('token', res.data)

        # Next we will assert that response is HTTP_400_BAD_REQUEST
        self.assertEqual(res.status_code, status.HTTP_400_BAD_REQUEST)

    # The first thing we're going to do is we're going to create a test
    # to make sure that authentication is required on the manage user
    # endpoint.
    def test_retrieve_user_unauthorized(self):
        """
        Test that authentication is required for the user.
        """
        res = self.client.post(ME_URL)

        self.assertEqual(res.status_code, status.HTTP_401_UNAUTHORIZED)

# We are going to make a PrivateUserAPITest class for authenticated test.
# Means test those require authentication will be in this class.
class PrivateUserAPITest(TestCase):
    """
    Test API requests that require authentication.
    """
    # Part of the setup, we're going to do the authentication
    # for each test that we do. So we don't need to basically set
    # the authentication for every single test we're just do the
    # setup and then that happens automatically before each test.
    def setup(self):
        """
        This function is run before every test from this
        class is run. So some times there are some set up
        task, that need to be done before running every
        test from this class.
        """
        self.user = create_user(
            email='mujahid7292@gmail.com',
            password='strongPassword123',
            name='Saifullah Al Mujahid'
        )
        # This above code create a valid user in our system, so
        # that other function from this class can access this
        # specific user.

        self.client = APIClient()
        # What this above line does is, it basically set
        # APIClient() to self, so that other function from
        # this class can access APIClient()
        self.client.force_authenticate(user=self.user)
        # force_authenticate() method is used to authenticate any
        # requests that the client makes with our above user.

    # Next thing we're going to do is add our retrieve profile
    # successful test. We're just going to test that we can
    # retrieve the profile of the logged in user.
    def test_retrieve_profile_success(self):
        """
        Test retrieving profile for logged in user.
        """
        # Then what we will do is we'll just make the request because
        # we've already authenticated in our setup so we don't need
        # to do that authentication in this function.
        res = self.client.post(ME_URL)

        self.assertEqual(res.status_code, status.HTTP_200_OK)

        # Then we're going to test that the user object returned is
        # what we expect.
        self.assertEqual(res.data, {
            'name': self.user.name,
            'email': self.user.email
        })

    # Next we're going to test that you cannot do a HTTP POST request
    # on the profile.
    def test_post_on_me_url_not_allowed(self):
        """
        Test that post is not allowed on the me url.
        """
        res = self.client.post(ME_URL, {})

        self.assertEqual(
            res.status_code,
            status.HTTP_405_METHOD_NOT_ALLOWED
        )

    # Next we are going to add our user profile update test.
    # So we're going to update the user via the API and we're
    # going to test that the updates worked.
    def test_update_user_profile(self):
        """
        Test updating the user profile for authenticated user
        works.
        """
        payload = {
            'name': 'New Name',
            'password': 'newPassword123'
        }

        # Now we will update our user
        res = self.client.patch(ME_URL, payload)

        self.user.refresh_from_db()
        # This above code will refresh our existing user.

        # Now we will check that our user is updated or not.
        self.assertEqual(self.user.name, payload['name'])
        self.assertTrue(self.user.check_password(payload['password']))
        self.assertEqual(res.status_code, status.HTTP_200_OK)


Now go back to our terminal & run the test .
<br/><b>`$ sudo docker-compose run --rm app sh -c "python manage.py test && flake8"`</b><br/>

### Video: 48. Add manage user endpoint

Now we can actually create our manage user endpoint. We're going to use our existing user serializer but we're going to add an additional function to the serializer for updating our user object. We're also going to add a custom view using the retrieve update API view from the Django rest framework generic API view options. So let's go ahead and implement this.Head over to the `user/views.py` file.

In [ ]:
                                                                                    #user/views.py
from rest_framework import generics, authentication, permissions
from rest_framework.authtoken.views import ObtainAuthToken
from rest_framework.settings import api_settings
from user.serializers import UserSerializer, AuthTokenSerializer

# We're going to create a new view and that view are going to inherit
# from create API view, that comes with the Django rest framework.
# So this is a view that's pre-made for us that allows us to easily
# make a API that creates an object in a database using the serialize
# that we're going to provide.
class CreateUserView(generics.CreateAPIView):
    """
    Create a new user in the system.
    """
    serializer_class = UserSerializer


class CreateAuthToken(ObtainAuthToken):
    """
    Create a new auth token for user.
    """
    serializer_class = AuthTokenSerializer
    renderer_classes = api_settings.DEFAULT_RENDERER_CLASSES

class ManageUserView(generics.RetrieveUpdateAPIView):
    """
    Manage the authenticated user.
    """
    serializer_class = UserSerializer
    # Below this we're going to add two more class variables for
    # authentication and permission. So authentication is the
    # mechanism by which the authentication happens so this could
    # be cookie authentication or we're going to use is token
    # authentication. And the permissions are the level of access
    # that the user has, so the only permission we're going to add is
    # that the user must be authenticated to use the API they don't
    # have to have any special permissions they just have to be
    # logged in.
    authentication_classes = (authentication.TokenAuthentication,)
    permission_classes = (permissions.IsAuthenticated,)

    # Finally we need to add a get_object() function to our API view.
    # So typically what would happen with an API view is you would
    # link it to a model and it could retrieve the item and you
    # would retrieve data based models. In this case we're going to
    # just get the model for the logged in user. So we're going to
    # override the get object and we're just going to return the user
    # that is authenticated.
    def get_object(self):
        """
        Retrieve and return authenticated user.
        """
        return self.request.user
        # So when the get_object() is called the request will have the
        # user attached to it because of the 'authentication_classes'.
        # So because we have the 'authentication_classes' that takes
        # care of getting the authenticated user and assigning it to
        # request.

In [ ]:
                                                                                #user/serializers.py
from django.contrib.auth import get_user_model, authenticate
# authenticate = it's a Django helper command for working with
# the Django authentication system. So you simply pass in the
# username and password and you can authenticate a request.
from django.utils.translation import ugettext_lazy as _
from rest_framework import serializers


class UserSerializer(serializers.ModelSerializer):
    """
    Serializer for the user object.
    """
    class Meta:
        model = get_user_model()
        # This above line will give us the model, which we will
        # serialize in this class.

        # Next you can specify the fields that you want to include
        # in serializer so these are the fields that are going to
        # be converted to and from json when we make our HTTP POST
        # request. So those basically are the fields that we want
        # to make accessible in the API either to read or write.
        fields = ('email', 'password', 'name')
        # These are the three fields that we're going to accept
        # when we create users.

        # So finally we're going to add something called extra
        # keyword args and what this does is it allows us to configure
        # a few extra settings in our model serializer and what we're
        # going to use this for is to ensure that the password is write
        # only and that the minimum required length is 5 characters.
        extra_kwargs = {
            'password': {
                'write_only': True,
                'min_length': 5
                }
            }

    def create(self, validated_data):
        """
        Create a new user with an encrypted password and
        return it.
        """
        return get_user_model().objects.create_user(**validated_data)

    # Now we will create update() function. The purpose of this function
    # is, we want to make sure the user password is set using the set
    # password function instead of just setting it to whichever value
    # is provided.
    def update(self, instance, validated_data):
        """
        Update a user, setting the password correctly and return it.
        """
        # instance = This is going to be the model instance that is
        # linked to our ModelSerializer and that's going to be our
        # user object.
        # validated_data = {'email', 'password', 'name'}

        # First we will remove the password from the validated
        # data. We do that using the dictionary pop function.
        password = validated_data.pop('password', None)
        # The reason we provide None here is because with the pop
        # function you must provide a default value. What pop does
        # is it basically is very similar to get except after its
        # retrieved it, it will remove it from the original dictionary.
        user = super().update(instance, validated_data)
        # What we're doing here is, super() will call the ModelSerializer
        # update() functions. So it will call the default function. So
        # we can make use of all the functionality that's included in
        # the default one whilst extending it slightly to customize it
        # for our needs.

        # Next we're going to set the password,if the user provided
        # a password.
        if password:
            user.set_password(password)
            user.save()
            
        return user

class AuthTokenSerializer(serializers.Serializer):
        """
        Serializer for the user authentication object.
        """
        email = serializers.CharField()
        password = serializers.CharField(
            style={'input_type': 'password'},
            trim_whitespace=False
        )

        # This validate function is called when we validate our serializer.
        # This validation function will check whether inputs are all correct,
        # like whether email is CharField etc And as part of the validation
        # function we are also going to validate that the authentication
        # credentials are correct.
        def validate(self, attrs):
            """
            Validate and authenticate the user.
            """
            # attrs = This is a dictionary, which contain our passed in
            # email and password. So this attrs parameter here is basically
            # just every field that makes up our serializer. So any field
            # that makes up a serializer, it will get passed into the
            # validate function here as this dictionary and then we can
            # retrieve the fields via this attributes and we can
            # then validate whether we want to pass this validation or we want
            # to fail the validation. Now we will rtrieve those email and
            # password.
            email = attrs.get('email')
            password = attrs.get('password')

            # Now we will authenticate our request.
            user = authenticate(
                request=self.context.get('request'),
                username=email,
                password=password
            )

            # If our above authentication fail, then
            if not user:
                # Means our authentication failed.
                msg = _('Unable to authenticate with provided credentials')
                raise serializers.ValidationError(
                    msg,
                    code='authentication'
                )
            # Now we will return our 'attrs' dictionary by
            # inserting our authenticated user.
            attrs['user'] = user
            return attrs


In [ ]:
                                                                                    #user/urls.py
from django.urls import path
from user import views

# Then we can define our app name and the app name is set to
# help identify which app we're creating the URL from.
app_name = 'user'

urlpatterns = [
    path('create/', views.CreateUserView.as_view(), name='create'),
    path('token/', views.CreateAuthToken.as_view(), name='token'),
    path('me/', views.ManageUserView.as_view(), name='me'),
]


Now go back to our terminal & run the test .
<br/><b>`$ sudo docker-compose run --rm app sh -c "python manage.py test && flake8"`</b><br/>

In [ ]:
48.AddManageUserEndPoint

In [ ]:
### Video: 25. Implement custom user model<b>``</b>
<b>`$ `</b>
<br/><b>`$ `</b>